In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.callbacks import TensorBoard
from keras.utils.np_utils import to_categorical
from keras.models import load_model

import time
import numpy as np
import random
import os
from math import *

from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import LambdaCallback

from my_to_midi import *

In [6]:
model_path = '/Users/mac/Desktop/Brain/MusicResearch/Music_Text_Generation/Model_logdir/Bach_batchS1024_epochs200_units512_maxL32_step1_embeddingL8_2018-07-30_102638/epoch200.h5'
model = load_model(model_path)

In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
def generate_music(epoch, text, diversity, start_index, is_train=False):
    print_fn('----- diversity: %.1f' % diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print_fn('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(generate_length):
        x_pred = np.zeros((1, maxlen, len(chars) * embedding_length))
        for t, char in enumerate(sentence):
            for idx in range(embedding_length):
                x_pred[0, t, idx * embedding_length + char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

    if is_train:
        log_name = "epoch%d_train_diversity%02d" % (epoch + 1, int(diversity * 10))
    else:
        if start_index == 0:
            log_name = "epoch%d_first_diversity%02d" % (epoch + 1, int(diversity * 10))
        else:
            log_name = "epoch%d_random_diversity%02d" % (epoch + 1, int(diversity * 10))

    text_log_path = os.path.join(text_log_dir, log_name + ".txt")
    with open(text_log_path, "w") as text_log_file:
        text_log_file.write(generated + "\n")
        text_log_file.close()

    print_fn("Write %s.txt to %s" % (log_name, text_log_dir))

    events = text_to_events(generated)
    events_to_midi('basic_rnn', events, midi_log_dir, log_name)

    print_fn("Write %s.midi to %s" % (log_name, midi_log_dir))

    model_name = "epoch%d.h5" % (epoch+1)
    model_path = os.path.join(model_log_dir, model_name)
    model.save(model_path)
    print_fn("Save model %s.h5 to %s" % (model_name, model_log_dir))

In [ ]:
def baseline_music(epoch, text, start_index, is_train=False):
    print_fn('----- baseline')

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence

    generated += eval_text[start_index + maxlen: min(len(text), start_index + maxlen + generate_length)]
    sys.stdout.write(generated)

    if is_train:
        log_name = "epoch%d_train_baseline" % (epoch + 1)
    else:
        if start_index == 0:
            log_name = "epoch%d_first_baseline" % (epoch + 1)
        else:
            log_name = "epoch%d_random_baseline" % (epoch + 1)

    text_log_path = os.path.join(text_log_dir, log_name + ".txt")
    with open(text_log_path, "w") as text_log_file:
        text_log_file.write(generated + "\n")
        text_log_file.close()

    print_fn("Write %s.txt to %s" % (log_name, text_log_dir))

    events = text_to_events(generated)
    events_to_midi('basic_rnn', events, midi_log_dir, log_name)

    print_fn("Write %s.midi to %s" % (log_name, midi_log_dir))

In [5]:
def generate_from_model(epoch, logs):
    print_fn("")
    print_fn('----- Generating Music from model: test')

    start_index = random.randint(0, len(eval_text) - maxlen - 1)

    baseline_music(epoch=epoch, text=eval_text, start_index=0)
    baseline_music(epoch=epoch, text=eval_text, start_index=start_index)
    baseline_music(epoch=epoch, text=train_text, start_index=start_index, is_train=True)

    for diversity in [0.2, 0.5, 0.8, 1.0, 1.2]:
        generate_music(epoch=epoch, text=eval_text, diversity=diversity, start_index=0)
        generate_music(epoch=epoch, text=eval_text, diversity=diversity, start_index=start_index)
        generate_music(epoch=epoch, text=train_text, diversity=diversity, start_index=start_index, is_train=True)

print_fn("*"*20+exp_name+"*"*20)
print_fn('x_train shape:'+str(np.shape(x_train)) )
print_fn('y_train shape:'+str(np.shape(y_train)) )



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32, 16)            4880      
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1083392   
_________________________________________________________________
dense_2 (Dense)              (None, 38)                19494     
_________________________________________________________________
activation_1 (Activation)    (None, 38)                0         
Total params: 1,107,766
Trainable params: 1,107,766
Non-trainable params: 0
_________________________________________________________________


In [9]:
import re

exp_name = 'Bach_batchS1024_epochs200_units512_maxL128_step1_embeddingL1_2018-07-30_061429'

exp_pat = r"([a-zA-Z]*)_batchS(\d*)_epochs(\d*)_units(\d*)_denseS(\d*)_maxL(\d*)_step(\d*)_embeddingL(\d*)_(.*)"
exp_mat = re.match(exp_pat, exp_name)

if exp_mat:
    batch_size = exp_mat.group(1)
    epochs = exp_mat.group(2)
    units = exp_mat.group(3)
    dense_size = exp_mat.group(4)
    maxlen = exp_mat.group(5)
    step = exp_mat.group(6)
    embedding_length = exp_mat.group(7)
    date_and_time = exp_mat.group(8)
else:
    print("No match!")


No match!
